In [ ]:
from typing import Annotated, Dict, List
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
import gradio as gradio
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
import requests
from bs4 import BeautifulSoup

In [ ]:



class State(BaseModel):
    user_prompt: str
    mermaid_code: str = ""
    validation_result: Dict = {}
    description: str = ""
    diagram_type: str = ""
    diagram_type_context: str = ""
    iteration_count: int = 0
    max_iterations: int = 3
    errors: List[str] = []
    syntax_rules: Dict = {}

In [ ]:
from enum import Enum, auto

class DiagramType:
    SEQUENCE_DIAGRAM = "sequenceDiagram"
    FLOWCHART = "flowchart"
    CLASS_DIAGRAM = "classDiagram"
    STATE_DIAGRAM = "stateDiagram"
    ER_DIAGRAM = "erDiagram"
    GANTT = "gantt"
    PIE = "pie"
    QUADRANT = "quadrantChart"
    REQUIREMENT = "requirementDiagram"
    GITGRAPH = "gitGraph"
    MINDMAP = "mindmap"
    TIMELINE = "timeline"
    ZENUML = "zenuml"
    SANKEY = "sankey"
    

In [14]:
def generate_diagram(state: State) -> Dict:
    llm = ChatOpenAI(
        base_url="http://localhost:1234/v1",
        model="deepseek-coder-v2-lite-instruct", 
        api_key="sk-1234",
        temperature=0
    )
    
    prompt = f"""You are a Mermaid diagram expert. Generate a valid Mermaid diagram based on the user's request.

Syntax Reference for {state.diagram_type}:
{state.diagram_type_context}

User Request: {state.user_prompt}
{f"Description: {state.description}" if state.description else ""}

Generate ONLY the Mermaid diagram code. Start with the diagram type declaration.
Do not include markdown code blocks, explanations, or any other text.
Just the raw Mermaid syntax."""
    
    try:
        response = llm.invoke(prompt)
        mermaid_code = response.content.strip()
        
        # Καθάρισμα από markdown code blocks αν υπάρχουν
        if mermaid_code.startswith("```"):
            lines = mermaid_code.split("\n")
            mermaid_code = "\n".join(lines[1:-1]) if len(lines) > 2 else mermaid_code
        

        return {
            "mermaid_code": mermaid_code,
            "iteration_count": state.iteration_count + 1
        }
        
    except Exception as e:
        return {
            "errors": state.errors + [f"Failed to generate diagram: {str(e)}"],
            "iteration_count": state.iteration_count + 1
        }
    



# Helpers

In [ ]:
def fetch_page_content(url: str) -> str:
    """Fetch and extract text content from a web page."""
    response = requests.get(url)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Remove script and style elements
    for script in soup(["script", "style", "nav", "footer", "header"]):
        script.decompose()
    
    # Get text content
    text = soup.get_text()
    
    # Clean up whitespace
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    
    return text

In [ ]:
def get_summarized_syntax(content: str, diagram_type: DiagramType) -> str:
    """Send content to LLM for summarization of syntax rules."""
    llm = ChatOpenAI(
        base_url="http://localhost:1234/v1",
        model="deepseek-coder-v2-lite-instruct", 
        api_key="sk-1234",
        temperature=0
    )
    print(diagram_type)
    prompt = f"""You are a technical documentation assistant. 
Please summarize the Mermaid.js syntax rules for {diagram_type} diagrams from the following documentation.

Focus on:
- Basic syntax structure
- Key commands and keywords
- Common patterns and examples
- Important rules and constraints

Keep the summary concise but complete enough to be used as context for diagram generation.

Documentation content:
{content}

Provide a clear, structured summary:"""
    
    response = llm.invoke(prompt)
    print(response)
    return response.content

# Retrieve Syntax

In [ ]:
def diagram_syntax_reference(state: State) -> Dict:
    """
    Fetch and summarize Mermaid diagram syntax reference.
    
    Args:
        diagram_type: The type of diagram to get syntax for
        
    Returns:
        Summarized syntax rules as a string
    """
    # Mapping για specific documentation URLs
    diagram_type = state.diagram_type
    url_mapping = {
        DiagramType.SEQUENCE_DIAGRAM: "https://mermaid.js.org/syntax/sequenceDiagram.html",
        DiagramType.FLOWCHART: "https://mermaid.js.org/syntax/flowchart.html",
        DiagramType.CLASS_DIAGRAM: "https://mermaid.js.org/syntax/classDiagram.html",
        DiagramType.STATE_DIAGRAM: "https://mermaid.js.org/syntax/stateDiagram.html",
        DiagramType.ER_DIAGRAM: "https://mermaid.js.org/syntax/entityRelationshipDiagram.html",
        DiagramType.GANTT: "https://mermaid.js.org/syntax/gantt.html",
        DiagramType.PIE: "https://mermaid.js.org/syntax/pie.html",
        # Προσθήκη και άλλων URLs όπως χρειάζεται
    }
    
    # Πάρε το specific URL ή fallback στο general reference
    url = url_mapping.get(
        diagram_type, 
        "https://mermaid.js.org/intro/syntax-reference.html"
    )
    
    try:
        # Fetch το περιεχόμενο
        content = fetch_page_content(url)

        
        # Summarize με LLM
        summary = get_summarized_syntax(content, diagram_type)
        return {
            "diagram_type_context": summary,
            "diagram_type": diagram_type
        }
        
    except Exception as e:
        raise Exception(f"Failed to fetch or summarize syntax reference: {str(e)}")

# Validatediagram

In [ ]:
def validate_syntax(code: str) -> dict:
    pass

# get diagram

In [ ]:
def describe_diagram(code: str) -> str:
    return """
# Program Analysis: Program

## 1. Architectural Overview
**Program Type**: Application  
**Primary Purpose**: Mainframe job execution system with custom screen rendering for NETSOL logon interface
**Structural Complexity**: 19 components, 18 relationships

## 2. Component Architecture

### Core Components
| Component | Type | Role | Dependencies |
|-----------|------|------|-------------|
| JOB | Function | Job execution control | main |
| DATETIME | Function | System date/time formatting | TIME, GETMAIN, FREEMAIN |
| INQUIRE | Function | Terminal information retrieval | TK4MDEV |
| WRITE | Function | Screen output rendering | TK4MLOG |
| IDCAMS | Function | Dataset delete processing | SYS1.UMODMAC |
| IEBCOPY | Function | Dataset copy processing | SYS1.MACLIB |
| IEBUPDTE | Function | Dataset update processing | SYS1.UMODMAC |
| IFOX00 | Function | Assembly processing | System libraries |
| IEWL | Function | Link editing | SYS1.VTAMLIB |

### Execution Flow
```
[JOB]
├── [IDCAMS] → DELETE processing
├── [IEBCOPY] → COPY processing  
├── [IEBUPDTE] → UPDATE processing
├── [IFOX00] → ASSEMBLY processing
├── [IEWL] → LINK processing
├── [DATETIME] → Date/time formatting
├── [INQUIRE] → Terminal query
└── [WRITE] → Screen output
```

## 3. System Integration

### External Dependencies
**Services**: DELETE, COPY, ADD, CHANGE, WRITE, INQUIRE  
**Data Sources**: SYS1.UMODMAC, SYS1.MACLIB, SYS1.VTAMLIB  
**Platform Services**: TIME, GETMAIN, FREEMAIN

### Data Flow
**Inputs**: System date/time, terminal device information, source libraries (SYS1.UMODMAC, SYS1.MACLIB)  
**Processing**: Date/time conversion, terminal inquiry, dataset operations (delete/copy/update)  
**Outputs**: Formatted screen data (TK4MLOG), compiled modules (SYS1.VTAMLIB), updated libraries (SYS1.UMODMAC)

## 4. Technical Patterns

### Architectural Patterns
- **Batch Processing Pipeline**: Sequential execution of system utilities (IDCAMS→IEBCOPY→IEBUPDTE→IFOX00→IEWL)
- **Screen-Oriented Architecture**: Dedicated components for terminal interaction (INQUIRE, WRITE, WCC, SBA)
- **Resource Management**: Explicit memory allocation/deallocation (GETMAIN/FREEMAIN) in DATETIME routine

### Implementation Patterns
- **Macro-Based Screen Definition**: Uses WCC (Write Control Character) and SBA (Set Buffer Address) for 3270 screen control
- **System Service Abstraction**: Encapsulates platform services (TIME, GETMAIN) within DATETIME function
- **Conditional Processing**: Implements branching logic (BNE NOUSS, B SETLEAP, B CALCMON) for error handling and calculations

## 5. Quality Assessment

### Structural Integrity
- **Cohesion**: High functional cohesion within utility components, but mixed concerns in main control flow
- **Coupling**: Moderate data coupling (6) with clear resource dependencies between job steps
- **Complexity**: Moderate cyclomatic complexity (8) with well-defined control flow and error paths

### Recommendations
- **Symbol Classification Refinement**: Correct misclassification of macros (UMODMAC, WCC, SBA) as functions to improve analysis accuracy
- **Control Flow Enhancement**: Add missing edges for DATETIME routine loops (SCANMON) and conditional branches
- **Resource Dependency Mapping**: Expand data flow analysis to include all work areas (WORKAREA, SARESULT) and their usage patterns
    """

# build state

In [ ]:
def agent_builder():
    builder = StateGraph(State)

    builder.add_node('diagram_syntax_reference', diagram_syntax_reference)
    builder.add_node('generate_diagram', generate_diagram)

    builder.add_edge(START, 'diagram_syntax_reference')
    builder.add_edge('diagram_syntax_reference', 'generate_diagram')
    builder.add_edge('generate_diagram', END)

    return builder.compile()
     

# Generate Diagram

In [15]:
# Initial state
initial_state = State(
    user_prompt="Create a sequence diagram for user authentication flow with login, verification, and token generation",
    diagram_type=DiagramType.SEQUENCE_DIAGRAM,

)

agent = agent_builder()

result = agent.invoke(initial_state)
print(result)

if result.errors:
    print("Errors encountered:")
    for error in result.errors:
        print(f"  - {error}")


sequenceDiagram
content=' ### Summary of Mermaid.js Syntax Rules for Sequence Diagrams\n\n#### Basic Structure\n- **Participants**: Participants (actors) are defined implicitly or explicitly using keywords like `participant`, `actor`, etc. They appear in the order they are declared.\n- **Ordering**: The order of participants can be specified manually if needed.\n- **Groups**: Actors can be grouped vertically with optional colors and labels using `box` syntax.\n\n#### Key Commands and Keywords\n- **Participants**: \n  - `participant`: Defines a participant in the diagram.\n  - `actor`: Alternative to `participant`.\n- **Grouping**: \n  - `box`: Groups actors with optional color and label.\n- **Messages**: \n  - Arrows represent messages: `->`, `--`, `-->`, `-->>`, etc., with options for solid, dotted, or bidirectional arrows.\n- **Activations**: Actors can be activated/deactivated using `activate` and `deactivate`.\n- **Notes**: Notes can be added over actors or between participants.\n-

AttributeError: 'dict' object has no attribute 'errors'

In [ ]:
syntax_ref = diagram_syntax_reference(DiagramType.SEQUENCE_DIAGRAM)
print(syntax_ref)